In [1]:
"""Script converts all mseed files into numpy array. Each array is detrended by fitting a linear function and subtracting that line to remove the DC offset."""

'Script converts all mseed files into numpy array. Each array is detrended by fitting a linear function and subtracting that line to remove the DC offset.'

In [2]:
import obspy

import numpy as np
import pandas as pd
from glob import glob
import os
from os.path import join, basename, dirname, exists
import re
from fastparquet import write

In [3]:
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
ac_calib = 8.2928e-05
name_dic = {'be4':'lower','a3m':'upper','ad8':'horizontal'}

In [5]:
local_data_dir = '/home/zacharykeskinen/Documents/infrasound/data/banner/infrasound/processed'
target_dir = '/home/zacharykeskinen/Documents/infrasound/array_data'
assert exists(target_dir)

In [25]:
fps = glob(join(local_data_dir, '*'))
fps = [f for f in fps if '.1.' not in f]
array_names = np.unique([basename(f)[2:5] for f in fps])
assert set(array_names) == set(name_dic.keys()), 'Missing an array in dict?'
for ext, desc in name_dic.items():
    array_dir = join(target_dir, f'{ext}-{desc}')
    os.makedirs(array_dir, exist_ok= True)
    array_fps = sorted([f for f in fps if ext in f])
    if ext != 'ad8':
        for i, fp in enumerate(array_fps):
            if i < 4:
                tr = obspy.read(fp)[0]
                tr.detrend("linear")
                arr = np.array(tr.data * ac_calib)
                t = [datetime.fromtimestamp(t) for t in tr.times("timestamp")]
                stats = tr.stats
                res = pd.DataFrame(arr, index = t, columns = ['pa'])
                res.index = res.index.tz_localize('US/Mountain').tz_convert('UTC')

                ## Parquet
                channel = int(stats.channel.replace('p','')) + 1
                write(join(array_dir, f'{stats.starttime.date}_c{channel}.parq'), res, compression = 'GZIP')

RuntimeError: Compression 'SNAPPY' not available.  Options: ['BROTLI', 'GZIP', 'UNCOMPRESSED']

In [21]:
res = pd.read_parquet('/home/zacharykeskinen/Documents/infrasound/array_data/be4-lower/2021-12-02_c1.parq')

In [22]:
res

,pa
index,
2021-12-02 22:20:13.975000+00:00,-11.916051
2021-12-02 22:20:13.980000+00:00,-11.902609
2021-12-02 22:20:13.985000+00:00,-11.908074
2021-12-02 22:20:13.990000+00:00,-11.922164
2021-12-02 22:20:13.995000+00:00,-11.921659
...,...
2021-12-02 23:59:59.975000+00:00,2.179475
2021-12-02 23:59:59.980000+00:00,2.177409
2021-12-02 23:59:59.985000+00:00,2.178578


In [17]:
res.index.tz_localize('US/Mountain').tz_convert('UTC')

DatetimeIndex(['2022-01-21 19:10:57.975000+00:00',
               '2022-01-21 19:10:57.980000+00:00',
               '2022-01-21 19:10:57.985000+00:00',
               '2022-01-21 19:10:57.990000+00:00',
               '2022-01-21 19:10:57.995000+00:00',
                      '2022-01-21 19:10:58+00:00',
               '2022-01-21 19:10:58.005000+00:00',
               '2022-01-21 19:10:58.010000+00:00',
               '2022-01-21 19:10:58.015000+00:00',
               '2022-01-21 19:10:58.020000+00:00',
               ...
               '2022-01-21 23:59:59.950000+00:00',
               '2022-01-21 23:59:59.955000+00:00',
               '2022-01-21 23:59:59.960000+00:00',
               '2022-01-21 23:59:59.965000+00:00',
               '2022-01-21 23:59:59.970000+00:00',
               '2022-01-21 23:59:59.975000+00:00',
               '2022-01-21 23:59:59.980000+00:00',
               '2022-01-21 23:59:59.985000+00:00',
               '2022-01-21 23:59:59.990000+00:00',
            

In [12]:
from obspy.core import UTCDateTime

In [13]:
tr.stats.starttime

2022-01-21T19:10:57.975000Z

In [8]:
tr.times(reftime=stats.starttime)

array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
         1.73420100e+04,   1.73420150e+04,   1.73420200e+04])

In [9]:
tr.times(reftime=UTCDateTime(tr.stats.starttime))

array([  0.00000000e+00,   5.00000000e-03,   1.00000000e-02, ...,
         1.73420100e+04,   1.73420150e+04,   1.73420200e+04])

In [14]:
tr.times(type = 'utcdatetime')

array([UTCDateTime(2022, 1, 21, 19, 10, 57, 975000),
       UTCDateTime(2022, 1, 21, 19, 10, 57, 980000),
       UTCDateTime(2022, 1, 21, 19, 10, 57, 985000), ...,
       UTCDateTime(2022, 1, 21, 23, 59, 59, 985000),
       UTCDateTime(2022, 1, 21, 23, 59, 59, 990000),
       UTCDateTime(2022, 1, 21, 23, 59, 59, 995000)], dtype=object)

In [11]:
tr.stats

         network: 
         station: c0A3M
        location: 
         channel: p2
       starttime: 2022-01-21T19:10:57.975000Z
         endtime: 2022-01-21T23:59:59.995000Z
   sampling_rate: 200.0
           delta: 0.005
            npts: 3468405
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 1027, 'encoding': 'STEIM1', 'byteorder': '<', 'record_length': 4096, 'filesize': 4206592})
      processing: ["ObsPy 1.2.2: detrend(options={}::type='linear')"]